# Page Visits Funnel

In this project, you’ll be analyzing data for Cool T-Shirts Inc. Your job is to build a funnel, which is a description of how many people continue to the next step of a multi-step process. This funnel will describe how users move through their website and eventually purchase a t-shirt. You’ll use merges in Pandas to create this funnel from multiple tables.

Your job is to build a funnel, which is a description of how many people continue to the next step of a multi-step process.

In this case, our funnel is going to describe the following process:

1. A user visits CoolTShirts.com
2. A user adds a t-shirt to their cart
3. A user clicks "checkout"
4. A user actually purchases a t-shirt



In [2]:
import pandas as pd

visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

print(visits.head(),'\n')
print(cart.head(),'\n')
print(checkout.head(),'\n')
print(purchase.head())

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00 

                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00 

                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd472

In [6]:
# Combine visits and cart using a left merge.
merged = pd.merge(visits, cart, how='left')
merged.shape[0] # 2052 rows

2052

In [12]:
# How many of the timestamps are null for the column cart_time?
null_time_in_cart = cart['cart_time'].isnull().count()
null_time_in_cart # 400 null values for cart time

# What do these null rows mean? 
# => no record exists for that cart (zero cart time)

400

In [24]:
# What percent of users who visited Cool T-Shirts Inc. ended up not placing a t-shirt in their cart?
# Note: To calculate percentages, it will be helpful to turn either the numerator or the denominator 
# into a float, by using float(), with the number to convert passed in as input. Otherwise, Python 
# will use integer division, which truncates decimal points.
visited_no_order = (merged[merged.cart_time.isnull()].count() / merged.count()) * 100
print(visited_no_order['visit_time']) #=> 80.5%

80.50682261208577


In [26]:
cart_checkout = pd.merge(cart, checkout, how='left')
percent_checkouts_from_cart = (cart_checkout[cart_checkout.cart_time.isnull()].count() / cart_checkout.count()) * 100
percent_checkouts_from_cart

user_id          0.0
cart_time        0.0
checkout_time    0.0
dtype: float64

In [27]:
# Merge all four steps of the funnel, in order, using a series of left merges. Save the results to the variable all_data.
all_data = visits.merge(cart,how='left').merge(checkout,how='left').merge(purchase,how='left')
# Examine the result using print and head.
all_data.head()

user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
4  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2 2017-08-20 08:31:00                 NaT                 NaT  
3 2017-08-20 08:49:00                 NaT                 NaT  
4                 NaT                 NaT                 NaT

In [35]:
# What percentage of users proceeded to checkout, but did not purchase a t-shirt?
checkout_no_purchase = all_data[all_data.purchase_time.isnull()].count() / all_data[~all_data.checkout_time.isnull()].count()
checkout_no_purchase

user_id          2.325980
visit_time       2.325980
cart_time        0.301471
checkout_time    0.147059
purchase_time    0.000000
dtype: float64